In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils as utils
from torch.utils.data import Dataset, DataLoader
import torchvision as vision
from torchvision import transforms, utils, datasets

import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os
from skimage import io, transform
from PIL import Image
import sys

In [2]:
categories = [x[0] for x in os.walk('ILSVRC/Data/CLS-LOC/train')] # obtain all subdirectories

In [3]:
categories = categories[1:] # eliminate root directory

In [4]:
categories = [path.split('/')[4] for path in categories] # obtain subfolder name

In [5]:
categoryMap = {i : j for j, i in enumerate(categories, 0)} # create category mapping

In [6]:
# instantiate data class

class ImageNetData(Dataset):
    def __init__(self, csv_file, root_dir, categoryMap, transform=None):
        """
        Args:
            csv_file (string): csv_file containing labels
            root_dir (string): directory containing all data
            transform (callable, optional): optional transforms to be applied to an image
        """
        # process labels
        self.trainLabels = pd.read_csv(csv_file) # read label file
        self.trainLabels['PredictionString'] = self.trainLabels['PredictionString'].str.split(n=1).str.get(0) # process so only category labels remain
        self.trainLabels['PredictionString'] = self.trainLabels['PredictionString'].map(categoryMap).astype(np.long) # map labels to category indices
        
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.trainLabels)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        img_folder = self.trainLabels['ImageId'].iloc[idx].split('_')[0]
        img_name = self.trainLabels['ImageId'].iloc[idx] + '.JPEG'
        img_path = os.path.join(self.root_dir, img_folder, img_name)
        
        image = Image.open(img_path)
        image = image.convert('RGB')
        label = np.array(self.trainLabels['PredictionString'].iloc[idx])
        
        if self.transform:
            image = self.transform(image)
        
        sample = {'image': image, 'label': label}
        
        return sample

In [7]:
# instantiate validation data class
class ImageNetValidation(Dataset):
    def __init__(self, csv_file, root_dir, categoryMap, transform=None):
        """
        Args:
            csv_file (string): csv_file containing labels
            root_dir (string): directory containing all data
            transform (callable, optional): optional transforms to be applied to an image
        """
        # process labels
        self.valLabels = pd.read_csv(csv_file) # read label file
        self.valLabels['PredictionString'] = self.valLabels['PredictionString'].str.split(n=1).str.get(0) # process so only category labels remain
        self.valLabels['PredictionString'] = self.valLabels['PredictionString'].map(categoryMap).astype(np.long) # map labels to category indices
        
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.valLabels)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        #img_folder = self.valLabels['ImageId'].iloc[idx].split('_')[0]
        img_name = self.valLabels['ImageId'].iloc[idx] + '.JPEG'
        img_path = os.path.join(self.root_dir, img_name)
        
        image = Image.open(img_path)
        image = image.convert('RGB')
        label = np.array(self.valLabels['PredictionString'].iloc[idx])
        
        if self.transform:
            image = self.transform(image)
        
        sample = {'image': image, 'label': label}
        
        return sample

imageData = ImageNetData(csv_file = 'LOC_train_solution.csv', root_dir = 'ILSVRC/Data/CLS-LOC/train')

fig = plt.figure()
plt.figure(figsize=(20, 20))

# show images
for i in range(4):
    sample = imageData[i]
    
    #print(i, sample['image'].shape, sample['label'].shape)
    
    ax = plt.subplot(4, 1, i+1)
    plt.tight_layout()
    ax.set_title('Sample {}'.format(i))
    ax.axis('off')
    plt.imshow(sample['image'])

valData = ImageNetValidation(csv_file = 'LOC_val_solution.csv', root_dir = 'ILSVRC/Data/CLS-LOC/val')

fig = plt.figure()
plt.figure(figsize=(20, 20))

# show images
for i in range(4):
    sample = valData[i]
    
    #print(i, sample['image'].shape, sample['label'].shape)
    
    ax = plt.subplot(4, 1, i+1)
    plt.tight_layout()
    ax.set_title('Sample {}'.format(i))
    ax.axis('off')
    plt.imshow(sample['image'])

In [8]:
# create image transformations
train_transform = transforms.Compose([transforms.Resize(size=(336, 336)),
                                      transforms.RandomRotation(degrees=20),
                                      transforms.RandomCrop(size=224),
                                      transforms.RandomVerticalFlip(p=0.5),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
                                      transforms.RandomErasing(scale=(0.02, 0.1))])

val_transform = transforms.Compose([transforms.Resize(size=(224, 224)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

# see the effects of transformation
#imageData = ImageNetData(csv_file = 'LOC_train_solution.csv', root_dir = 'ILSVRC/Data/CLS-LOC/train')

fig = plt.figure()
plt.figure(figsize=(20, 20))

# show images
for i in range(4):
    sample = imageData[i]
    
    transformed_image = train_transform(sample['image'])
    
    print(i, transformed_image.shape, sample['label'].shape)
    
    ax = plt.subplot(4, 1, i+1)
    plt.tight_layout()
    ax.set_title('Sample {}'.format(i))
    ax.axis('off')
    plt.imshow(transformed_image.numpy().transpose(1, 2, 0))

fig = plt.figure()
plt.figure(figsize=(20, 20))

# show images
for i in range(4):
    sample = valData[i]
    
    transformed_image = val_transform(sample['image'])
    
    print(i, transformed_image.shape, sample['label'].shape)
    
    ax = plt.subplot(4, 1, i+1)
    plt.tight_layout()
    ax.set_title('Sample {}'.format(i))
    ax.axis('off')
    plt.imshow(transformed_image.numpy().transpose(1, 2, 0))

trainLabelFile = 'LOC_train_solution.csv'
train_dir = 'ILSVRC/Data/CLS-LOC/train'
valLabelFile = 'LOC_val_solution.csv'
val_dir = 'ILSVRC/Data/CLS-LOC/val'
ImageNetDataset = ImageNetData(csv_file = trainLabelFile, root_dir = train_dir, transform = train_transform)
train_loader = DataLoader(ImageNetDataset, batch_size=512, shuffle=True, num_workers=16)
ImageNetVal = ImageNetValidation(csv_file = valLabelFile, root_dir = val_dir, transform = val_transform)
val_loader = DataLoader(ImageNetVal, batch_size=512, shuffle=True, num_workers=16)

for i in range(len(ImageNetDataset)):
    img = ImageNetDataset[i]
    if img['image'].shape[0] != 3:
        print("ERROR")


image_batch = next(iter(train_loader))
grid = utils.make_grid(image_batch['image'])
plt.figure(figsize=(20, 20))
plt.imshow(grid.numpy().transpose((1, 2, 0)))

val_batch = next(iter(val_loader))
grid = utils.make_grid(val_batch['image'])
plt.figure(figsize=(20, 20))
plt.imshow(grid.numpy().transpose((1, 2, 0)))

data, lab = image_batch
print(data)

batch = next(iter(train_loader))

len(batch['image'])

In [9]:
# define network
class deepNet(nn.Module):
    def __init__(self):
        super(deepNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 128, 3, stride=2)
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, 3)
        self.batchnorm4 = nn.BatchNorm2d(128)
        self.conv5 = nn.Conv2d(128, 384, 3, stride=2)
        self.inceptionA_batchnorm = nn.BatchNorm2d(384)
        self.inceptionA_conv11 = nn.Conv2d(384, 32, 1)
        self.inceptionA_batchnorm11 = nn.BatchNorm2d(32)
        self.inceptionA_conv12 = nn.Conv2d(384, 32, 1)
        self.inceptionA_batchnorm12 = nn.BatchNorm2d(32)
        self.inceptionA_conv13 = nn.Conv2d(384, 32, 1)
        self.inceptionA_batchnorm13 = nn.BatchNorm2d(32)
        self.inceptionA_conv31 = nn.Conv2d(32, 32, 3, padding=1)
        self.inceptionA_batchnorm31 = nn.BatchNorm2d(32)
        self.inceptionA_conv32 = nn.Conv2d(32, 48, 3, padding=1)
        self.inceptionA_batchnorm32 = nn.BatchNorm2d(48)
        self.inceptionA_conv33 = nn.Conv2d(48, 64, 3, padding=1)
        self.inceptionA_batchnorm33 = nn.BatchNorm2d(64)
        self.inceptionA_conv14 = nn.Conv2d(128, 384, 1)
        self.reductionA_conv6 = nn.Conv2d(384, 1154, 1)
        self.inceptionB_batchnorm = nn.BatchNorm2d(1154)
        self.inceptionB_conv11 = nn.Conv2d(1154, 192, 1)
        self.inceptionB_batchnorm11 = nn.BatchNorm2d(192)
        self.inceptionB_conv12 = nn.Conv2d(1154, 128, 1)
        self.inceptionB_batchnorm12 = nn.BatchNorm2d(128)
        self.inceptionB_conv71 = nn.Conv2d(128, 160, kernel_size=(1, 7), padding=(0, 3))
        self.inceptionB_batchnorm71 = nn.BatchNorm2d(160)
        self.inceptionB_conv72 = nn.Conv2d(160, 192, kernel_size=(7, 1), padding=(3, 0))
        self.inceptionB_batchnorm72 = nn.BatchNorm2d(192)
        self.inceptionB_conv13 = nn.Conv2d(384, 1154, 1)
        self.reductionB_conv7 = nn.Conv2d(1154, 2048, 1)
        self.inceptionC_batchnorm = nn.BatchNorm2d(2048)
        self.inceptionC_conv11 = nn.Conv2d(2048, 192, 1)
        self.inceptionC_batchnorm11 = nn.BatchNorm2d(192)
        self.inceptionC_conv12 = nn.Conv2d(2048, 192, 1)
        self.inceptionC_batchnorm12 = nn.BatchNorm2d(192)
        self.inceptionC_conv31 = nn.Conv2d(192, 224, kernel_size=(1, 3), padding=(0, 1))
        self.inceptionC_batchnorm31 = nn.BatchNorm2d(224)
        self.inceptionC_conv32 = nn.Conv2d(224, 256, kernel_size=(3, 1), padding=(1, 0))
        self.inceptionC_batchnorm32 = nn.BatchNorm2d(256)
        self.inceptionC_conv13 = nn.Conv2d(448, 2048, 1)
        self.batchnorm5 = nn.BatchNorm1d(2048)
        self.fc1000 = nn.Linear(2048, 1000)
        
    def inceptionA(self, x):
        x = F.relu(self.inceptionA_batchnorm(x))
        x1 = self.inceptionA_conv11(x)
        x1 = F.relu(self.inceptionA_batchnorm11(x1))
        x2 = self.inceptionA_conv12(x)
        x2 = F.relu(self.inceptionA_batchnorm12(x2))
        x2 = self.inceptionA_conv31(x2)
        x2 = F.relu(self.inceptionA_batchnorm31(x2))
        x3 = self.inceptionA_conv13(x)
        x3 = F.relu(self.inceptionA_batchnorm13(x3))
        x3 = self.inceptionA_conv32(x3)
        x3 = F.relu(self.inceptionA_batchnorm32(x3))
        x3 = self.inceptionA_conv33(x3)
        x3 = F.relu(self.inceptionA_batchnorm33(x3))
        x_concat = torch.cat((x1, x2, x3), 1)
        out = self.inceptionA_conv14(x_concat)
        
        return out
    
    def inceptionB(self, x):
        x = F.relu(self.inceptionB_batchnorm(x))
        x1 = self.inceptionB_conv11(x)
        x1 = F.relu(self.inceptionB_batchnorm11(x1))
        x2 = self.inceptionB_conv12(x)
        x2 = F.relu(self.inceptionB_batchnorm12(x2))
        x2 = self.inceptionB_conv71(x2)
        x2 = F.relu(self.inceptionB_batchnorm71(x2))
        x2 = self.inceptionB_conv72(x2)
        x2 = F.relu(self.inceptionB_batchnorm72(x2))
        x_concat = torch.cat((x1, x2), 1)
        out = self.inceptionB_conv13(x_concat)
        
        return out
    
    def inceptionC(self, x):
        x = F.relu(self.inceptionC_batchnorm(x))
        x1 = self.inceptionC_conv11(x)
        x1 = F.relu(self.inceptionC_batchnorm11(x1))
        x2 = self.inceptionC_conv12(x)
        x2 = F.relu(self.inceptionC_batchnorm12(x2))
        x2 = self.inceptionC_conv31(x2)
        x2 = F.relu(self.inceptionC_batchnorm31(x2))
        x2 = self.inceptionC_conv32(x2)
        x2 = F.relu(self.inceptionC_batchnorm32(x2))
        x_concat = torch.cat((x1, x2), 1)
        out = self.inceptionC_conv13(x_concat)
        
        return out
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.batchnorm1(x))
        x = self.conv2(x)
        x = F.relu(self.batchnorm2(x))
        x = self.conv3(x)
        x = F.relu(self.batchnorm3(x))
        x = self.conv4(x)
        x = F.relu(self.batchnorm4(x))
        x = self.conv5(x)
        x = F.max_pool2d(x, kernel_size=(2,2), stride=2)
        x = F.relu(self.inceptionA_batchnorm(x))
        x_res1 = self.inceptionA(x)
        x_res1 = x + x_res1
        x_res1 = self.reductionA_conv6(x_res1)
        x_res1 = F.max_pool2d(x_res1, kernel_size=(2,2), stride=2)
        x_res2 = self.inceptionB(x_res1)
        x_res2 = x_res2 + x_res1
        x_res2 = self.reductionB_conv7(x_res2)
        x_res2 = F.max_pool2d(x_res2, kernel_size=(2,2), stride=2)
        x_res3 = self.inceptionC(x_res2)
        x_res3 = x_res3 + x_res2
        x_res3 = F.max_pool2d(x_res3, kernel_size=(6,6))
        x_res3 = torch.flatten(x_res3, start_dim=1)
        x_res3 = F.relu(self.batchnorm5(x_res3))
        out = self.fc1000(x_res3)
        
        return out

In [10]:
# define parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 128
start_epoch = 0
best_val_acc = 0
lr = 0.001
save_dir = "Models"
val_conv_net = "Inception_Resnet_custom_bestval"
train_conv_net = "Inception_Resnet_custom_train"
Start_From_Checkpoint = 1

In [11]:
trainLabelFile = 'LOC_train_solution.csv'
train_dir = 'ILSVRC/Data/CLS-LOC/train'
valLabelFile = 'LOC_val_solution.csv'
val_dir = 'ILSVRC/Data/CLS-LOC/val'
ImageNetDataset = ImageNetData(csv_file = trainLabelFile, root_dir = train_dir, categoryMap = categoryMap, transform = train_transform)
train_loader = DataLoader(ImageNetDataset, batch_size=batch_size, shuffle=True, num_workers=8)
ImageNetVal = ImageNetValidation(csv_file = valLabelFile, root_dir = val_dir, categoryMap = categoryMap, transform = val_transform)
val_loader = DataLoader(ImageNetVal, batch_size=batch_size, shuffle=True, num_workers=8)

In [12]:
length_train = len(ImageNetDataset)
length_val = len(ImageNetVal)

In [13]:
SaveVal = os.path.join(save_dir, val_conv_net + '.pt')
SaveTrain = os.path.join(save_dir, train_conv_net + '.pt')

#Create the save directory if it does note exist
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

In [14]:
net = deepNet()
if (torch.cuda.device_count() > 1):
    print("Let's use ", torch.cuda.device_count(), " GPUs!\n")
    net = nn.DataParallel(net)

Let's use  2  GPUs!



In [15]:
# accuracy calculator
def calculate_accuracy(fx, y):
    preds = fx.max(1, keepdim=True)[1]
    correct = preds.eq(y.view_as(preds)).sum()
    acc = correct.float()/preds.shape[0]
    return acc

In [16]:
# define loss and optimiser, and scheduler (for lr adjustment)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.3)

# DEFINE TRAIN FUNCTION

In [17]:
# define train function
def train(net, device, criterion, optimizer, train_loader, epoch, length_train):
    epoch_loss = 0.0
    epoch_acc = 0.0
    
    for i, data in enumerate(train_loader, 0):
        inp = data['image']
        lab = data['label']
        
        inp = inp.to(device)
        lab = lab.to(device)
        
        optimizer.zero_grad()
        
        # forward propagation
        out = net.forward(inp)
        loss = criterion(out, lab)
        acc = calculate_accuracy(out, lab)
        
        # backward propagation
        loss.backward()
        
        # update network
        optimizer.step()
        
        epoch_loss += loss
        epoch_acc += acc
        
        #print(f"TRAINING  |  ITER: {i+1:02}  |  LOSS: {loss.item():.3f}", end = '\r')
        print(f'EPOCH {epoch+1:02}  |  ITER {i+1:02}/{length_train//128+1:02}  |  Training loss {loss.item():.3f}  |  Training acc {acc*100:.3f}', end='\r')
        
    return epoch_loss/len(train_loader), epoch_acc/len(train_loader)

# DEFINE EVALUATION FUNCTION

In [18]:
# define evaluation function
def evaluate(net, device, criterion, test_loader, length_test):
    epoch_loss = 0.0
    epoch_acc = 0.0
    
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            inp = data['image']
            lab = data['label']
            inp = inp.to(device)
            lab = lab.to(device)
            
            # forward propagation
            out = net(inp)
            loss = criterion(out, lab)
            acc = calculate_accuracy(out, lab)
            
            epoch_loss += loss
            epoch_acc += acc
                
            print(f"EVALUATION  |  ITER: {i+1:02}/{length_test//128+1:02}  |  LOSS: {loss.item():.3f}", end = '\r')
            
    return epoch_loss/len(test_loader), epoch_acc/len(test_loader)

In [19]:
train_loss_log = []
train_acc_log = []
validation_loss_log = []
validation_acc_log = []

In [20]:
#Load Checkpoint
if Start_From_Checkpoint:
    #Check if checkpoint exists
    if os.path.isfile(SaveTrain):
        #load Checkpoint
        check_point = torch.load(SaveTrain)
        #Checkpoint is saved as a python dictionary
        #https://www.w3schools.com/python/python_dictionaries.asp
        #here we unpack the dictionary to get our previous training states
        net.load_state_dict(check_point['model_state_dict'])
        optimizer = optim.Adam(net.parameters(), lr=lr)
        optimizer.load_state_dict(check_point['optimizer_state_dict'])
        for state in optimizer.state.values():
            for k, v in state.items():
                if torch.is_tensor(v):
                    state[k] = v.cuda()
        start_epoch = check_point['epoch']
        train_loss_log = check_point['train_loss']
        train_acc_log = check_point['train_acc']
        print("Training loaded, starting from epoch:", start_epoch)
    else:
        #Raise Error if it does not exist
        raise ValueError("Training network does not exist")
    if os.path.isfile(SaveVal):
        #load Checkpoint
        check_point = torch.load(SaveVal)
        #Checkpoint is saved as a python dictionary
        #https://www.w3schools.com/python/python_dictionaries.asp
        #here we unpack the dictionary to get our previous training states
        #net.load_state_dict(check_point['model_state_dict'])
        #optimizer.load_state_dict(check_point['optimizer_state_dict'])
        #start_epoch = check_point['epoch']
        best_valid_acc = check_point['val_acc']
        #print("Checkpoint loaded, starting from epoch:", start_epoch)
    else:
        raise ValueError("Validation network does not exist")
else:
    #If checkpoint does exist and Start_From_Checkpoint = False
    #Raise an error to prevent accidental overwriting
    if (os.path.isfile(SaveTrain) or os.path.isfile(SaveVal)):
        raise ValueError("Warning Checkpoint exists")
    else:
        print("Starting from scratch")

Training loaded, starting from epoch: 74


In [21]:
net.to(device)

DataParallel(
  (module): deepNet(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (batchnorm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (batchnorm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(32, 128, kernel_size=(3, 3), stride=(2, 2))
    (batchnorm3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (batchnorm4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv5): Conv2d(128, 384, kernel_size=(3, 3), stride=(2, 2))
    (inceptionA_batchnorm): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (inceptionA_conv11): Conv2d(384, 32, kernel_size=(1, 1), stride=(1, 1))
    (inceptionA_batchnorm11): BatchNorm2d(32, eps=1e-05, momentum=0

In [ ]:
# TRAIN THE NETWORK

for epoch in range(start_epoch, 500):
    
    # run training for 1 epoch
    train_loss, train_acc = train(net, device, criterion, optimizer, train_loader, epoch, length_train)
    
    train_loss_log.append(train_loss)
    train_acc_log.append(train_acc)
    
    #scheduler.step()
    
    # run evaluation every 20 epochs
    if not (epoch+1)%20:
        val_loss, val_acc = evaluate(net, device, criterion, val_loader, length_val)
        
        # save best validation model
        if (val_acc > best_val_acc):
            best_val_acc = val_acc
            torch.save({
                'epoch':                    epoch,
                'model_state_dict':         net.state_dict(),
                'optimizer_state_dict':     optimizer.state_dict(),
                'val_acc':                val_acc
            }, SaveVal)
        # save training model
        torch.save({
            'epoch':                    epoch,
            'model_state_dict':         net.state_dict(),
            'optimizer_state_dict':     optimizer.state_dict(),
            'train_acc':                train_acc_log,
            'train_loss':               train_loss_log
        }, SaveTrain)
        
    #print(f'EPOCH {epoch+1:02}  |  Training loss {train_loss:.3f}  |  Training acc {train_acc*100:.3f}  |  Validation loss {val_loss:.3f}  |  Validation acc {val_acc*100:.3f}', end='\r')

x = torch.randn(32, 2, 3)
y = torch.randn(64, 2, 3)

torch.cat((x, y), 0)

train_batch = next(iter(train_loader))

train_batch['label']

train_batch['label'].shape

val_loss, val_acc = evaluate(net, device, criterion, val_loader, len(val_loader))
    
validation_loss_log.append(val_loss)
    
validation_acc_log.append(val_acc)

if (val_acc > best_val_acc):
    best_val_acc = val_acc
    print("Saving model", end='\r')
    torch.save({
        'epoch':                    20,
        'model_state_dict':         net.state_dict(),
        'optimizer_state_dict':     optimizer.state_dict(),
        'val_acc':                val_acc
    }, SaveVal)

print("Saving model", end='\r')
torch.save({
    'epoch':                    74,
    'model_state_dict':         net.state_dict(),
    'optimizer_state_dict':     optimizer.state_dict(),
    'train_acc':                train_acc_log,
    'train_loss':               train_loss_log
}, SaveTrain)